In [ ]:
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,Bidirectional,Flatten,TimeDistributed, Conv1D, MaxPooling1D, BatchNormalization

import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.seasonal import STL
import os
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import tensorflow as tf
print("Number GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Cleaning

In [ ]:
df = pd.read_csv("/home/ug2019/eee/19085023/Techfest/trainset.csv", index_col = "Region_Name")
dfr = df.T
dfr.rename(columns={"Region_Name":"Month"}, inplace=True)
dfr.insert(0, "Months", np.arange(1, 73))
dfr.reset_index(inplace=True)
dfr.drop(columns=["index"], inplace=True)
plt.figure(figsize=(10,4))
plt.plot(dfr["Region 1015"])
for year in range(0,72, 6):
    plt.axvline(year, color='k', linestyle='--', alpha=0.5)
mon = ["Month {}".format(i) for i in range(1, 73)]
dict = {"Month":mon}
lilclin = pd.DataFrame(dict)
for i in range(1, 3916):
    print(i)
    region = "Region " + str(i)
    stl = STL(dfr[region], period = 6)
    result = stl.fit()
    seasonal, trend, resid = result.seasonal, result.trend, result.resid
    estimated = trend + seasonal
    lilclin[region] = estimated
    # plt.figure(figsize=(12,4))
    # plt.plot(dfr[region])
    # plt.plot(estimated)
lilclin.to_csv("/home/ug2019/eee/19085023/Techfest/lilclin.csv")
# lilclin
plt.figure(figsize=(8,6))

# plt.subplot(4,1,1)
# plt.plot(dfr["Region 475"])
# plt.title('Original Series', fontsize=16)

# plt.subplot(4,1,2)
# plt.plot(trend)
# plt.title('Trend', fontsize=16)

# plt.subplot(4,1,3)
# plt.plot(seasonal)
# plt.title('Seasonal', fontsize=16)

# plt.subplot(4,1,4)
# plt.plot(resid)
# plt.title('Residual', fontsize=16)

plt.tight_layout()
estimated = trend + seasonal
plt.figure(figsize=(12,4))
plt.plot(dfr["Region 1015"])
plt.plot(estimated)
resid_mu = resid.mean()
resid_dev = resid.std()

lower = resid_mu - 1.5*resid_dev
upper = resid_mu + 1.5*resid_dev
plt.figure(figsize=(10,4))
plt.plot(resid)

plt.fill_between((1, 72), lower, upper, color='g', alpha=0.25, linestyle='--', linewidth=2)
plt.xlim(1, 72)
lilclin["Region 475"]



In [ ]:
for i in range(1, 3916):
    reg = "Region " + str(i)
    lilclin[reg] = np.where(lilclin[reg]<lilclin[reg].quantile(0.25), lilclin[reg].quantile(0.25), lilclin[reg])
    lilclin[reg] = np.where(lilclin[reg]>lilclin[reg].quantile(0.75), lilclin[reg].quantile(0.75), lilclin[reg])
lilclin.drop(columns="Unnamed: 0", inplace=True)
lilclin.to_csv("/home/ug2019/eee/19085023/Techfest/quartlilclin.csv")

In [ ]:
add = "/home/ug2019/eee/19085023/Techfest/quartlilclin.csv" #supercomputer address

df = pd.read_csv(add, index_col="Month")

# no = np.array(dfr["Region " + str(i+1)])

# dev = np.array(df).flatten().std()
# avg = np.array(df).flatten().mean()

# df-=avg
# df/=dev

# print(avg)
# print(dev)

avg, dev = [], []

n = np.array(df)

# avg = np.array(df)

for i in range(1,3916):

    avgg = df.iloc[:,i].mean()
    stdd = df.iloc[:,i].std()
    
    df.iloc[:,i] = (df.iloc[:,i]-avgg)/stdd
    
    avg.append(avgg)
    dev.append(stdd)


In [ ]:
# print(avg)

In [ ]:
dfr=df

In [ ]:
def prepare_data(timeseries_data, n_features):
	X, y = [],[]
	for i in range(len(timeseries_data)):
		end_ix = i + n_features
		if end_ix > len(timeseries_data) - 1:
			break
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
dict = {"Month":np.arange(73, 88)}
fincsv = pd.DataFrame(dict)
fincsv.set_index("Month", inplace=True)

In [ ]:
def create_model():
    model = Sequential()
    # model.add(TimeDistributed(Conv1D(filters=128, kernel_size=2, activation='relu'), input_shape=(None, 3, 2)))
    # model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    # model.add(BatchNormalization())
    # model.add(TimeDistributed(Flatten()))
    model.add((LSTM(1024, activation='tanh',input_shape=(None, 6) , return_sequences=True)))
    model.add((LSTM(1024, activation='tanh', return_sequences=True)))
    model.add((LSTM(512, activation='tanh', return_sequences=True)))
    model.add((LSTM(256, activation='tanh')))
    model.add((LSTM(256, activation='tanh')))
    model.add((LSTM(256, activation='tanh')))
    model.add((LSTM(128, activation='tanh')))
    
    model.add(Dense(128, activation='relu', kernel_regularizer = keras.regularizers.l2(0.01)))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu', kernel_regularizer = keras.regularizers.l2(0.01)))
    
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu', kernel_regularizer = keras.regularizers.l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_regularizer = keras.regularizers.l2(0.01)))
    # model.add(Dense(64, activation='sigmoid'))
    model.add(Dense(1))
    opt = keras.optimizers.Adam(learning_rate = 0.02)
    opt2 = keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9)
    
    model.compile(optimizer = 'adam', loss='mape')
    
    return model

In [ ]:
m1 = create_model()
m1.summary()

In [ ]:
cor = dfr.iloc[:,1:].corr(method = "pearson")
corr = np.array(cor)
cor.shape

In [ ]:
# len(cor.iloc[0, :])
# cor

In [ ]:
ind = [False]*len(cor.iloc[0, :])

def find_corr_cols(l_thres, u_thresh):
    # ind = [False]*len(cor.iloc[0, :])
    # thresh = 0.9
    related = []
    unrelated = []
    # flag=1
    for i in range(0, len(cor.iloc[0, :])):
        if ind[i] == False:
            list = []
            list.append(i+1)
            for j in range(i+1, len(cor.iloc[0, :])):
                if l_thres < corr[i, j] and corr[i, j] <= u_thresh and ind[j] == False:
                    list.append(j+1)
                    # flag=0
                    ind[i] = True
                    ind[j] = True
                    print("related is ",i+1)
            if len(list) == 1:
                print("unrelated is ",i+1)
                unrelated.append(i+1)
                # ind[i] = False
            else:
                print("related is ",i+1)
                related.append(list)
            # if flag == 1 : 
                

    return related, unrelated



related9, unrelated9 = find_corr_cols(0.95, 1)
related8, unrelated8 = find_corr_cols(0.85, 0.95)
related7, unrelated7 = find_corr_cols(0.75, 0.85)
# unrelated = unrelated8
print(len(related9)+len(related8)+len(related7))


In [ ]:
print(len(related9))
print(len(related8))
print(len(unrelated9))
s = 0
for i in range(len(related9)):
    s = s + len(related9[i])

for i in range(len(related8)):
    s = s + len(related8[i])

for i in range(len(related7)):
    s = s + len(related7[i])
print(3915 - s)
# np.array(related9[0])

In [ ]:
allrelated = related9 + related8 + related7 

for uns in unrelated7:
    allrelated.append([uns])
s=0
for i in range(len(allrelated)):
    s = s + len(allrelated[i])
s
len(allrelated)
# allrelated[-7:]

In [ ]:
# ls_correg = [[2, 12]]
np.random.shuffle(allrelated)
trainx= []
trainy = []
for ind,pee in enumerate(allrelated):
    
#     if(ind > 200):break

    region = "Region " + str(pee[0])
    
    X, y = prepare_data(dfr[region], 6)
        
    X_train = X[:]
    
    y_train = y[:]
    
    # X_train = X_train.reshape((X_train.shape[0], 6))
    
    trainx.append(X_train)
    trainy.append(y_train)
    mont = 15
    n_features = 1
print(len(trainx))
trainx = np.array(trainx)
trainy = np.array(trainy)

In [ ]:
# trainx = trainx.reshape(672*66, 6)
# trainy = trainy.flatten()

In [ ]:
print(trainy.shape)
print(trainx.shape)

In [ ]:
trainx = trainx.reshape((trainx.shape[0], 66, 6))

In [ ]:

# model = load_model(str(add + "reved.h5"))



# history = univ.fit(
#     trainx, trainy, epochs=10000, 
#     verbose=2,
#     shuffle=True,

#     # workers = 16,
#     # use_multiprocessing = True,

# )

# save_path = add+str("reved.h5")
# univ.save(save_path)


In [ ]:
# mirrored_strategy = tf.distribute.MirroredStrategy()
# with mirrored_strategy.scope():
univ = create_model()

# import matplotlib.pyplot as plt

history = univ.fit(
    trainx, trainy, epochs = 400000, 
    verbose=2,
    shuffle=True,
    workers = 16,
    use_multiprocessing = True,

)

save_path = "reved.h5"
univ.save(save_path)



In [ ]:

# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()


In [ ]:
dict = {"Month":np.arange(73, 88)}
fincsv = pd.DataFrame(dict)
fincsv.set_index("Month", inplace=True)

In [ ]:
st = 0
en = 3915

In [ ]:
for k in range(st,en):

    region = "Region " + str(1+k)
    
    regar = np.array(dfr[region])
    print(regar)
    print(k)
    
    for i in range(15):
                
        test = regar[-6:].reshape((1, 1, 6))
        
        print(test)
        
        yhat = univ.predict(
            test,
            verbose=1,
            # use_multiprocessing = True
        )
        
        print(yhat)
        
        print("old -", regar.shape)
        
        regar = np.append(regar, yhat)
        
        print("new -", regar.shape)
    
    print(regar[-15:])

    fincsv[region] = regar[-15:]*dev[k]+avg[k]


In [ ]:
fincsv.to_csv(str(1+st)+'-'+str(en)+'x.csv')

fincsv


# ddd = pd.read_csv(str(st)+'-'+str(en)+'.csv')


In [ ]:
# fincsv.to_csv("l4.csv")
# fincsv

In [ ]:
# import pandas as pd

# ddd = pd.read_csv(str(st)+'-'+str(en)+'.csv')


In [ ]:
# ddd.head()

In [ ]:
# ddd.shape

In [ ]:
# f = fincsv

In [ ]:
# f.head(3)

In [ ]:
# a = pd.read_csv('clirimasort.csv')

# a.iloc[:,2000:] -= .3
# a.iloc[:,1:2000] -= .1

# a.iloc[:,1:] *= .4

# a.to_csv('f2.csv')